In [1]:
from keras import models
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Input
import pandas as pd
import numpy as np
import random

# Модель

In [2]:
LAST_SIGMOID = False
LAST_SIGMOID_K = 6

In [3]:
data = pd.read_csv("Elementary.csv", sep=';', index_col=0)
data

,Название,Поколение,Верх/низ,Электрический заряд,Спин,Масса (МэВ),Логарифм массы,Sigmoid массы,Частица/Античастица,Заряд
№,,,,,,,,,,
1,Верхний u-кварк,0.333333,1,0.666667,0.5,2.300000,0.832909,0.568967,1,0.666667
2,Нижний d-кварк,0.333333,-1,-0.333333,0.5,4.800000,1.568616,0.627819,1,0.333333
3,Очарованный c-кварк,0.666667,1,0.666667,0.5,1275.000000,7.150701,0.915566,1,0.666667
4,Странный s-кварк,0.666667,-1,-0.333333,0.5,95.000000,4.553877,0.820238,1,0.333333
5,Истинный t-кварк,1.000000,1,0.666667,0.5,174340.000000,12.068763,0.982414,1,0.666667
6,Прелестный b-кварк,1.000000,-1,-0.333333,0.5,4180.000000,8.338067,0.941550,1,0.333333
7,Верхний u-антикварк,0.333333,1,-0.666667,0.5,2.300000,0.832909,0.568967,-1,0.666667
8,Нижний d-антикварк,0.333333,-1,0.333333,0.5,4.800000,1.568616,0.627819,-1,0.333333
9,Очарованный c-антикварк,0.666667,1,-0.666667,0.5,1275.000000,7.150701,0.915566,-1,0.666667


In [4]:
#train_x = data[['Поколение', 'Верх/низ', 'Заряд', 'Спин', 'Частица/Античастица']].to_numpy()
train_x = data[['Поколение', 'Верх/низ', 'Электрический заряд', 'Спин', 'Частица/Античастица']].to_numpy()
if LAST_SIGMOID:
    train_y = data['Sigmoid массы'].to_numpy()
else:
    train_y = data['Логарифм массы'].to_numpy()

In [5]:
model = models.Sequential()
model.add(Dense(6, activation="sigmoid", input_shape=(train_x.shape[1],)))
model.add(Dense(1, activation="linear"))
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 36        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


# Обучение

In [6]:
callback = EarlyStopping(monitor='loss', min_delta=0.0001, patience=28*3, verbose=0, mode='auto', restore_best_weights=False)

In [7]:
%%time
batch = 1
for ind in range(0, 100):
    results = model.fit(train_x, train_y, epochs=500, batch_size=batch, shuffle=True, callbacks=[callback], verbose=0)
    print(ind+1, "Train energy:", np.mean(results.history["mean_absolute_error"]), f"batch={batch}")

1 Train energy: 4.270150733947754 batch=1
2 Train energy: 3.833469702720642 batch=1
3 Train energy: 3.3946334896087644 batch=1
4 Train energy: 2.769610702037811 batch=1
5 Train energy: 2.0485532400608064 batch=1
6 Train energy: 1.361910320520401 batch=1
7 Train energy: 0.8543950507640838 batch=1
8 Train energy: 0.500869429051876 batch=1
9 Train energy: 0.3466846495270729 batch=1
10 Train energy: 0.2854709050655365 batch=1
11 Train energy: 0.24439746862649916 batch=1
12 Train energy: 0.21911288458108902 batch=1
13 Train energy: 0.2018749270439148 batch=1
14 Train energy: 0.19282856011390687 batch=1
15 Train energy: 0.18889028465168345 batch=1
16 Train energy: 0.18733330133714174 batch=1
17 Train energy: 0.18449847631490052 batch=1
18 Train energy: 0.18078638379832349 batch=1
19 Train energy: 0.17771303461117 batch=1
20 Train energy: 0.17290437792542726 batch=1
21 Train energy: 0.16837982710348356 batch=1
22 Train energy: 0.16448095871342552 batch=1
23 Train energy: 0.16116868222461028 b

In [8]:
%%time
for ind in range(100):
    #batch = round(8 - 7/200 * ind)
    batch = random.randint(1, 8)
    results = model.fit(train_x, train_y, epochs=500, batch_size=batch, shuffle=True, callbacks=[callback], verbose=0)
    print(ind+1, "Train energy:", np.mean(results.history["mean_absolute_error"]), f"batch={batch}")

1 Train energy: 0.008030348810656318 batch=7
2 Train energy: 0.008000277810017851 batch=5
3 Train energy: 0.017761021403267102 batch=1
4 Train energy: 0.008760082979448313 batch=3
5 Train energy: 0.008173735258991227 batch=4
6 Train energy: 0.008154182453804157 batch=4
7 Train energy: 0.007924020969692398 batch=6
8 Train energy: 0.009146825766519588 batch=4
9 Train energy: 0.013786914451143792 batch=2
10 Train energy: 0.008012381426113494 batch=5
11 Train energy: 0.01009309313753072 batch=3
12 Train energy: 0.0075676861328675465 batch=7
13 Train energy: 0.008311975889784448 batch=5
14 Train energy: 0.007823675153229166 batch=6
15 Train energy: 0.009729626561131547 batch=4
16 Train energy: 0.012403145339339972 batch=3
17 Train energy: 0.007165913661832318 batch=7
18 Train energy: 0.007127972790861831 batch=7
19 Train energy: 0.016286858454249896 batch=1
20 Train energy: 0.007090569372572329 batch=8
21 Train energy: 0.009934388232581757 batch=2
22 Train energy: 0.00732316673897645 batch=

In [9]:
%%time
for ind in range(0, 100):
    batch = round(8 - 7/100 * ind)
    #batch = random.randint(1, 8)
    results = model.fit(train_x, train_y, epochs=500, batch_size=batch, shuffle=True, callbacks=[callback], verbose=0)
    print(ind+1, "Train energy:", np.mean(results.history["mean_absolute_error"]), f"batch={batch}")

1 Train energy: 0.0026299992028404684 batch=8
2 Train energy: 0.0026667080627863896 batch=8
3 Train energy: 0.0026325886548661135 batch=8
4 Train energy: 0.0026977698784321547 batch=8
5 Train energy: 0.0027665300623458977 batch=8
6 Train energy: 0.0028945493128369835 batch=8
7 Train energy: 0.0033366998219314745 batch=8
8 Train energy: 0.003758551933638313 batch=8
9 Train energy: 0.002758998325204148 batch=7
10 Train energy: 0.0035353618180927106 batch=7
11 Train energy: 0.0037237737382597785 batch=7
12 Train energy: 0.003595635563354282 batch=7
13 Train energy: 0.0038585686974008295 batch=7
14 Train energy: 0.00323751203934936 batch=7
15 Train energy: 0.004125005447798792 batch=7
16 Train energy: 0.003060640909654253 batch=7
17 Train energy: 0.0036006977096857394 batch=7
18 Train energy: 0.0030334303374676143 batch=7
19 Train energy: 0.0026944533469812834 batch=7
20 Train energy: 0.0040288648828316264 batch=7
21 Train energy: 0.0024301384219571074 batch=7
22 Train energy: 0.0029105174

In [10]:
%%script false
batch = 1
for ind in range(0, 100):
    results = model.fit(train_x, train_y, epochs=500, batch_size=batch, shuffle=True, callbacks=[callback], verbose=0)
    print(ind+1, "Train energy:", np.mean(results.history["mean_absolute_error"]), f"batch={batch}")
    
for ind in range(100):
    #batch = round(8 - 7/200 * ind)
    batch = random.randint(1, 8)
    results = model.fit(train_x, train_y, epochs=500, batch_size=batch, shuffle=True, callbacks=[callback], verbose=0)
    print(ind+1, "Train energy:", np.mean(results.history["mean_absolute_error"]), f"batch={batch}")
    
for ind in range(0, 100):
    batch = round(8 - 7/100 * ind)
    #batch = random.randint(1, 8)
    results = model.fit(train_x, train_y, epochs=500, batch_size=batch, shuffle=True, callbacks=[callback], verbose=0)
    print(ind+1, "Train energy:", np.mean(results.history["mean_absolute_error"]), f"batch={batch}")

Couldn't find program: 'false'


# Предсказание

In [25]:
%%script false
model.load_weights("model_6_sigmoid_6_1.h5")

In [26]:
model.layers[0].get_weights()

[array([[  1.7499053 ,  -7.6262784 ,   5.8975005 ,  -9.942805  ,
           5.1365113 ,  -4.985766  ],
        [  4.8297153 ,   5.8151317 ,   2.7630317 ,   8.385034  ,
           6.2767267 ,   5.683667  ],
        [ 14.113029  ,  -0.5249054 , -11.281317  ,   0.033202  ,
         -15.851465  ,   2.8652427 ],
        [ -1.8415352 ,   1.5278744 ,  -2.9268699 ,   4.243051  ,
          -3.044032  ,   1.852331  ],
        [  0.51657367,  -0.49792564,  -0.06669668,  -0.42695317,
          -1.1724058 ,   2.4816856 ]], dtype=float32),
 array([ 0.91280234, -2.3067098 ,  0.37946227, -0.48429763, -0.8316452 ,
        -0.02940624], dtype=float32)]

In [27]:
model.layers[1].get_weights()

[array([[19.207735 ],
        [-9.56473  ],
        [10.283778 ],
        [ 1.5448728],
        [ 8.964401 ],
        [-5.620888 ]], dtype=float32),
 array([-7.7471204], dtype=float32)]

In [28]:
pred_y = model.predict(train_x).ravel()
mtrx = np.transpose(np.vstack((pred_y, train_y, pred_y-train_y)))
mtrx

array([[ 8.32591534e-01,  8.32909123e-01, -3.17589339e-04],
       [ 1.56734896e+00,  1.56861592e+00, -1.26696094e-03],
       [ 7.14534712e+00,  7.15070146e+00, -5.35433962e-03],
       [ 4.55426645e+00,  4.55387689e+00,  3.89560789e-04],
       [ 1.20491314e+01,  1.20687627e+01, -1.96312966e-02],
       [ 8.33945656e+00,  8.33806653e+00,  1.39003223e-03],
       [ 8.27679157e-01,  8.32909123e-01, -5.22996574e-03],
       [ 1.57350683e+00,  1.56861592e+00,  4.89091412e-03],
       [ 7.14413309e+00,  7.15070146e+00, -6.56836703e-03],
       [ 4.55560255e+00,  4.55387689e+00,  1.72565851e-03],
       [ 1.20611629e+01,  1.20687627e+01, -7.59974139e-03],
       [ 8.33426094e+00,  8.33806653e+00, -3.80558545e-03],
       [-6.70382023e-01, -6.71387744e-01,  1.00572114e-03],
       [-6.11913013e+00, -6.11929792e+00,  1.67784417e-04],
       [ 4.66086340e+00,  4.66021101e+00,  6.52392506e-04],
       [-1.77812052e+00, -1.77195684e+00, -6.16367573e-03],
       [ 7.48292303e+00,  7.48258053e+00

In [29]:
if LAST_SIGMOID:
    pred_y2 = np.exp(LAST_SIGMOID_K * np.arctanh(2 * pred_y - 1))
    train_y2 = np.exp(LAST_SIGMOID_K * np.arctanh(2 * train_y - 1))
else:
    pred_y2 = np.exp(pred_y)
    train_y2 = np.exp(train_y)
mtrx2 = np.transpose(np.vstack((pred_y2, train_y2, pred_y2-train_y2)))
mtrx2

array([[ 2.29926968e+00,  2.30000000e+00, -7.30323941e-04],
       [ 4.79392242e+00,  4.80000000e+00, -6.07757610e-03],
       [ 1.26819141e+03,  1.27500000e+03, -6.80859427e+00],
       [ 9.50370178e+01,  9.50000000e+01,  3.70177843e-02],
       [ 1.70950859e+05,  1.74339999e+05, -3.38913983e+03],
       [ 4.18581445e+03,  4.18000000e+03,  5.81445111e+00],
       [ 2.28800249e+00,  2.30000000e+00, -1.19975092e-02],
       [ 4.82353401e+00,  4.80000000e+00,  2.35340114e-02],
       [ 1.26665283e+03,  1.27500000e+03, -8.34716849e+00],
       [ 9.51640778e+01,  9.50000000e+01,  1.64077721e-01],
       [ 1.73020078e+05,  1.74339999e+05, -1.31992108e+03],
       [ 4.16412305e+03,  4.18000000e+03, -1.58769551e+01],
       [ 5.11513114e-01,  5.10998950e-01,  5.14164193e-04],
       [ 2.20036926e-03,  2.20000000e-03,  3.69258321e-07],
       [ 1.05727325e+02,  1.05658374e+02,  6.89509782e-02],
       [ 1.68955401e-01,  1.70000000e-01, -1.04459940e-03],
       [ 1.77742871e+03,  1.77682000e+03

# Сохранение

In [16]:
model.save_weights("model_6_sigmoid_6_1.h5")

In [31]:
df = pd.DataFrame(data=mtrx2, columns=['pred', 'real', 'delta'])
df.to_csv('pred.csv', sep=';')